## Resolución del desafio 1
Alumno: Juan Miguel Chunga

**Nota: Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

## Instalacion de paquetes y carga de modulos

In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Ejercicio 1

Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.

Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

## Avances parciales

* Se ha elaborado el set de entrenamiento
* Se han vectorizado los documentos
* Se piden seleccionar 5 documentos de manera aleatoria. Para ello, se obtendran 5 numeros de manera aleatoria. La semilla sera fija para garantizar repetitividad.

In [ ]:
# Obtencion de numeros al azar
import random

# Fijar la semilla para reproducibilidad
random.seed(42)

# Generar una lista de 5 números aleatorios entre 0 y el tamaño de X_train
random_indices = [random.randint(0, X_train.shape[0]-1) for _ in range(5)]
print(random_indices)
# Los documentos seran [10476, 1824, 409, 4506, 4012]

[10476, 1824, 409, 4506, 4012]


In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


Usando las herramientas anteriores, se diseñan funciones auxiliares para su ejecucion en los diferentes documentos.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def comparar_documentos(indice=0):
    """
    Compara un documento contra el resto usando similitud coseno.
    Devuelve:
      - Tipo del documento origen
      - Tabla con los 5 documentos más similares y sus clases
    """
    # Similitud coseno
    cossim = cosine_similarity(X_train[indice], X_train)[0]

    # Valores y posiciones ordenadas
    mostsim_idx = np.argsort(cossim)[::-1][1:6]
    cossim_vals = np.sort(cossim)[::-1][1:6]

    # Tipo de documento origen
    ori_type = newsgroups_train.target_names[y_train[indice]]

    # Crear DataFrame para ordenar los datos
    df = pd.DataFrame({
        "Indice documento": mostsim_idx,
        "Similitud coseno": cossim_vals,
        "Clase destino": [newsgroups_train.target_names[y_train[i]] for i in mostsim_idx]
    })

    return ori_type, df


In [ ]:
# Ejemplo de llamado a la funcion
ori,tabla=comparar_documentos(0)
print("Documento origen: ",ori)
print("Sobre las similitudes identificadas:")
print(tabla)

Documento origen:  rec.autos
Sobre las similitudes identificadas:
   Indice documento  Similitud coseno Clase destino
0              8013          0.305865     rec.autos
1              2554          0.292249     rec.autos
2              5553          0.280730     rec.autos
3              8266          0.280169     rec.autos
4              5282          0.276992     rec.autos


En este caso, se observa que el metodo funciona muy bien, dado que la clase de destino es del mismo tipo, aunque los valores de la similitud coseno son relativamente bajos.

## Documento de indice 10476

In [ ]:
ori,tabla=comparar_documentos(random_indices[0])
print("Documento origen: ",ori)
print("Sobre las similitudes identificadas:")
print(tabla)

Documento origen:  rec.sport.hockey
Sobre las similitudes identificadas:
   Indice documento  Similitud coseno          Clase destino
0              5064          0.225036       rec.sport.hockey
1              9623          0.217432  talk.politics.mideast
2             10575          0.216444              sci.crypt
3             10836          0.212603            alt.atheism
4              2350          0.211071              sci.crypt


En este caso, el tipo de origen es `rec.sport.hockey`. El documento mas cercano coincide en su tipo, mientras que los demas estan cerca en el espacio vectorial, pero parecen no tener una relacion directa en el tema. Ademas, los valores de la similitud coseno no parecen tener una relevancia alta.

## Documento 1824

In [ ]:
ori,tabla=comparar_documentos(random_indices[1])
print("Documento origen: ",ori)
print("Sobre las similitudes identificadas:")
print(tabla)

Documento origen:  comp.sys.mac.hardware
Sobre las similitudes identificadas:
   Indice documento  Similitud coseno          Clase destino
0              9921          0.354198  comp.sys.mac.hardware
1              6364          0.313191  comp.sys.mac.hardware
2              5509          0.304137  comp.sys.mac.hardware
3              2641          0.250383  comp.sys.mac.hardware
4              4359          0.241676  comp.sys.mac.hardware


En este caso, el tipo de origen es `comp.sys.mac.hardware`. Los 5 documentos mas cercanos coinciden en su tipo. Los valores de la similitud coseno no parecen tener una relevancia alta.

## Documento 409

In [ ]:
ori,tabla=comparar_documentos(random_indices[2])
print("Documento origen: ",ori)
print("Sobre las similitudes identificadas:")
print(tabla)

Documento origen:  comp.graphics
Sobre las similitudes identificadas:
   Indice documento  Similitud coseno  Clase destino
0              3444          0.230533  comp.graphics
1              5799          0.209073  comp.graphics
2              5905          0.198175  comp.graphics
3              1764          0.183850  comp.graphics
4              3364          0.165854  comp.graphics


En este caso, el tipo de origen es `comp.graphics`. Los 5 documentos mas cercanos coinciden en su tipo. Los valores de la similitud coseno no parecen tener una relevancia alta.

## Documento 4506

In [ ]:
ori,tabla=comparar_documentos(random_indices[3])
print("Documento origen: ",ori)
print("Sobre las similitudes identificadas:")
print(tabla)

Documento origen:  rec.autos
Sobre las similitudes identificadas:
   Indice documento  Similitud coseno          Clase destino
0              4211          0.189361        rec.motorcycles
1              5928          0.168237  comp.sys.mac.hardware
2              6224          0.158334              rec.autos
3              5171          0.157694              rec.autos
4              9491          0.152190              rec.autos


En este caso, el tipo de origen es `rec.autos`. El documento mas cercano no coincide en su tipo, no obstante, tiene mucha relacion. El segundo documento no aparenta tener una relacion directa, y los ultimos tres documentos son del mismo tipo. Nuevamente, los valores de la similitud coseno no parecen tener una relevancia alta.

## Documento 4012

In [ ]:
ori,tabla=comparar_documentos(random_indices[4])
print("Documento origen: ",ori)
print("Sobre las similitudes identificadas:")
print(tabla)

Documento origen:  rec.sport.hockey
Sobre las similitudes identificadas:
   Indice documento  Similitud coseno           Clase destino
0              6599          0.160013  soc.religion.christian
1             10644          0.142769        rec.sport.hockey
2              7478          0.135821        rec.sport.hockey
3              7308          0.131774        rec.sport.hockey
4             10792          0.130732      rec.sport.baseball


En este caso, el tipo de origen es `rec.sport.hockey`. El documento mas cercano no coincide en su tipo y no muestra relacion aparente. Los siguientes coinciden en tipo hasta el ultimo, que toca otro tipo de deporte. Nuevamente, los valores de la similitud coseno no parecen tener una relevancia alta.

Tras observar los experimentos realizados, podemos concluir que la similitud coseno no es suficiente por si misma para detectar documentos similares. Sin embargo, en algunos casos, si es un indicador adecuado de partida.

Complementado con otros metodos puede brindar un resultado adecuado.

## Ejercicio 2

Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

Para realizar esta actividad, creamos una funcion auxiliar

In [ ]:
def clasificar_zero_shot(vector_doc_prueba, vectores_entrenamiento, labels_entrenamiento):
    """
    Clasifica un documento de prueba utilizando un enfoque zero-shot
    basado en la similaridad coseno.

    Args:
        vector_doc_prueba: Representación vectorial del documento de prueba (matriz dispersa).
        vectores_entrenamiento: Matriz dispersa con los vectores de los documentos de entrenamiento.
        labels_entrenamiento: Etiquetas correspondientes a los documentos de entrenamiento.

    Returns:
        La etiqueta de clase predicha para el documento de prueba.
    """
    # Calcular la similaridad coseno entre el documento de prueba y todos los documentos de entrenamiento
    similitudes = cosine_similarity(vector_doc_prueba, vectores_entrenamiento)[0]

    # Encontrar el índice del documento de entrenamiento más similar
    indice_mas_similar = np.argmax(similitudes)

    # Devolver la etiqueta correspondiente al documento más similar
    return labels_entrenamiento[indice_mas_similar]


# Asegurar que X_test y y_test estén definidos transformando el conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

# Aplicar el clasificador zero-shot al conjunto de test
print("Aplicando el clasificador zero-shot al conjunto de prueba...")
y_pred_zero_shot = []

for i in range(X_test.shape[0]):
    # Clasificar cada documento de prueba individualmente
    etiqueta_predicha = clasificar_zero_shot(X_test[i], X_train, y_train)
    y_pred_zero_shot.append(etiqueta_predicha)

print("Clasificación zero-shot completa.")

# Convertir la lista a un array de NumPy para calcular el F1-score
y_pred_zero_shot = np.array(y_pred_zero_shot)

# Evaluar el desempeño del modelo
f1_zero_shot = f1_score(y_test, y_pred_zero_shot, average='macro')

print(f"El clasificador zero-shot obtuvo un F1-score (promedio macro) de {f1_zero_shot:.4f}.")

Aplicando el clasificador zero-shot al conjunto de prueba...
Clasificación zero-shot completa.
El clasificador zero-shot obtuvo un F1-score (promedio macro) de 0.5050.


## Resumen

Comparado con el F1-score del modelo Naïve Bayes (0.5854), el desempeño del clasificador zero-shot es menor.

Esto sugiere que la simple comparación de similaridad con documentos individuales no captura bien las distribuciones subyacentes de clases, a diferencia de modelos estadísticos como Naïve Bayes, especialmente considerando las bajas similitudes coseno observadas en el ejercicio previo.  

El carácter zero-shot proviene de que este método no entrena un modelo para aprender fronteras de decisión, sino que clasifica únicamente usando la similaridad bruta entre documentos.


## Principales hallazgos del análisis de datos
- Se implementó y evaluó un enfoque de clasificación *zero-shot*, el cual asigna a cada documento de prueba la etiqueta del documento de entrenamiento más similar según la similaridad coseno.
- El clasificador zero-shot alcanzó un F1-score (promedio macro) de **0.5050** en el conjunto de prueba.
- Este desempeño es inferior al obtenido con el modelo Naïve Bayes, que logró un F1-score de **0.5854**.

## Ejercicio 3

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
#ComplementNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

Vectorizar los documentos

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvect = TfidfVectorizer()#Vectorizacion por defecto

# Fit en train y transform en train
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

# Solo transform en test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target


Los documentos se han vectorizado. Ahora, se deben entrenar los modelos de `MultinomialNB` y `ComplementNB`.


In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# Definimos los dos modelos
mnb = MultinomialNB(alpha=1.0)
cnb = ComplementNB(alpha=1.0)

# Entrenamiento
mnb.fit(X_train, y_train)
cnb.fit(X_train, y_train)

# Predicción en test
y_pred_mnb = mnb.predict(X_test)
y_pred_cnb = cnb.predict(X_test)

# F1-score macro
f1_mnb = f1_score(y_test, y_pred_mnb, average="macro")
f1_cnb = f1_score(y_test, y_pred_cnb, average="macro")

print("MultinomialNB - F1 macro:", f1_mnb)
print("ComplementNB  - F1 macro:", f1_cnb)

MultinomialNB - F1 macro: 0.5854345727938506
ComplementNB  - F1 macro: 0.692953349950875


Se rehace el entrenamiento con diferentes configuraciones de alpha.

In [ ]:
for a in [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]:
    cnb = MultinomialNB(alpha=a)
    cnb.fit(X_train, y_train)
    pred = cnb.predict(X_test)
    print(f'Alpha: {a}, F1: {f1_score(y_test, pred, average="macro")}')

Alpha: 0.01, F1: 0.6828611295250568
Alpha: 0.1, F1: 0.6564514103512165
Alpha: 0.5, F1: 0.615341523969213
Alpha: 1.0, F1: 0.5854345727938506
Alpha: 2.0, F1: 0.53998304998563
Alpha: 5.0, F1: 0.46885579817739476


Probamos varias configuraciones

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

min_dfs     = [1, 2, 5]
max_dfs     = [0.9, 0.8, 0.7]
max_feats   = [None, 20000, 50000]
alphas      = [0.1, 0.5, 1.0, 2.0]

mejor_f1_mnb  = 0
mejor_conf_mnb = None

mejor_f1_cnb  = 0
mejor_conf_cnb = None

for min_df in min_dfs:
    for max_df in max_dfs:
        for max_features in max_feats:
            # nuevo vectorizador
            tfidfvect = TfidfVectorizer(
                min_df=min_df,
                max_df=max_df,
                max_features=max_features,
            )

            # vectorizar train y test
            X_train = tfidfvect.fit_transform(newsgroups_train.data)
            y_train = newsgroups_train.target

            X_test  = tfidfvect.transform(newsgroups_test.data)
            y_test  = newsgroups_test.target

            for alpha in alphas:
                # MultinomialNB
                mnb = MultinomialNB(alpha=alpha)
                mnb.fit(X_train, y_train)
                y_pred_mnb = mnb.predict(X_test)
                f1_mnb = f1_score(y_test, y_pred_mnb, average="macro")

                print(f"[MNB] min_df={min_df}, max_df={max_df}, "
                      f"max_features={max_features}, alpha={alpha} -> F1={f1_mnb:.4f}")

                if f1_mnb > mejor_f1_mnb:
                    mejor_f1_mnb = f1_mnb
                    mejor_conf_mnb = {
                        "min_df": min_df,
                        "max_df": max_df,
                        "max_features": max_features,
                        "alpha": alpha
                    }

                # ComplementNB
                cnb = ComplementNB(alpha=alpha)
                cnb.fit(X_train, y_train)
                y_pred_cnb = cnb.predict(X_test)
                f1_cnb = f1_score(y_test, y_pred_cnb, average="macro")

                print(f"[CNB] min_df={min_df}, max_df={max_df}, "
                      f"max_features={max_features}, alpha={alpha} -> F1={f1_cnb:.4f}")

                if f1_cnb > mejor_f1_cnb:
                    mejor_f1_cnb = f1_cnb
                    mejor_conf_cnb = {
                        "min_df": min_df,
                        "max_df": max_df,
                        "max_features": max_features,
                        "alpha": alpha
                    }

print("\nResultados:")
print("\nMultinomialNB:")
print("  config:", mejor_conf_mnb)
print("  F1-macro:", mejor_f1_mnb)

print("\nComplementNB:")
print("  config:", mejor_conf_cnb)
print("  F1-macro:", mejor_f1_cnb)


[MNB] min_df=1, max_df=0.9, max_features=None, alpha=0.1 -> F1=0.6565
[CNB] min_df=1, max_df=0.9, max_features=None, alpha=0.1 -> F1=0.6954
[MNB] min_df=1, max_df=0.9, max_features=None, alpha=0.5 -> F1=0.6153
[CNB] min_df=1, max_df=0.9, max_features=None, alpha=0.5 -> F1=0.6961
[MNB] min_df=1, max_df=0.9, max_features=None, alpha=1.0 -> F1=0.5854
[CNB] min_df=1, max_df=0.9, max_features=None, alpha=1.0 -> F1=0.6930
[MNB] min_df=1, max_df=0.9, max_features=None, alpha=2.0 -> F1=0.5400
[CNB] min_df=1, max_df=0.9, max_features=None, alpha=2.0 -> F1=0.6821
[MNB] min_df=1, max_df=0.9, max_features=20000, alpha=0.1 -> F1=0.6731
[CNB] min_df=1, max_df=0.9, max_features=20000, alpha=0.1 -> F1=0.6796
[MNB] min_df=1, max_df=0.9, max_features=20000, alpha=0.5 -> F1=0.6365
[CNB] min_df=1, max_df=0.9, max_features=20000, alpha=0.5 -> F1=0.6870
[MNB] min_df=1, max_df=0.9, max_features=20000, alpha=1.0 -> F1=0.6077
[CNB] min_df=1, max_df=0.9, max_features=20000, alpha=1.0 -> F1=0.6863
[MNB] min_df=1

De los experimentos realizados, respetando la restricción de no modificar el hiperparámetro ngram_range de los vectorizadores, se obtuvo que el mejor rendimiento lo obtuvo el ComplementNB, que supera por muy poco al MultinomialNB.

Si lo comparamos con el clasificador zero-shot que alcanzó un F1-score (promedio macro) de **0.5050** y el modelo Naïve Bayes, que logró un F1-score de **0.5854**; ambos modelos implementados obtuvieron un rendimiento superior.

## Ejercicio 4

Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

Se pide transponer la matriz documento-termino.

Al vectorizar, se obtiene: M (documento × término)

Se necesita al transponerla: Mᵀ (término × documento)

Se seleccionan 5 palabras al azar. Para ello, buscaremos las palabras mas frecuentes y seleccionaremos de manera manual las mas frecuentes que sean relevantes.

In [ ]:
newsgroups_train = fetch_20newsgroups(
    subset="train",
    remove=("headers", "footers", "quotes")  #menos basura
)
tfidfvect = TfidfVectorizer(
    stop_words="english",
    token_pattern=r"(?u)\b[a-zA-Z]{3,}\b",  # Opcion de filtrado para eliminar "basura"
    min_df=3,   # opcional pero recomendable
    max_df=0.85 # opcional
)

# Volvemos a vectorizar
X_train = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = tfidfvect.get_feature_names_out()

In [ ]:
term_sums = X_train.sum(axis=0)

# Convert the sparse matrix sum to a dense array for easier sorting
term_sums_dense = np.asarray(term_sums).flatten()

# Get the indices of the top 20 terms
top_20_indices = term_sums_dense.argsort()[-20:][::-1]

print("Los 20 términos más utilizados (según la suma de TF-IDF) son:")
for i, idx in enumerate(top_20_indices):
    print(f"{i+1}. {idx2word[idx]} (TF-IDF sum: {term_sums_dense[idx]:.2f})")

Los 20 términos más utilizados (según la suma de TF-IDF) son:
1. like (TF-IDF sum: 161.16)
2. just (TF-IDF sum: 160.61)
3. know (TF-IDF sum: 158.19)
4. don (TF-IDF sum: 155.35)
5. people (TF-IDF sum: 137.09)
6. does (TF-IDF sum: 135.22)
7. think (TF-IDF sum: 132.52)
8. use (TF-IDF sum: 119.58)
9. thanks (TF-IDF sum: 118.18)
10. good (TF-IDF sum: 114.15)
11. time (TF-IDF sum: 109.45)
12. new (TF-IDF sum: 103.67)
13. edu (TF-IDF sum: 91.82)
14. need (TF-IDF sum: 88.68)
15. god (TF-IDF sum: 88.29)
16. make (TF-IDF sum: 87.06)
17. windows (TF-IDF sum: 86.05)
18. way (TF-IDF sum: 85.62)
19. want (TF-IDF sum: 85.13)
20. problem (TF-IDF sum: 84.01)


In [ ]:
#Se seleccionan las siguientes palabras
selected_words=["god", "windows", "people", "problem", "time"]

In [ ]:
def print_word_indices(words_list, vocabulary_array):
    """
    Imprime palabra y el indice mas similar.

    Args:
        words_list (list): A list of words to find indices for.
        vocabulary_array (np.ndarray): An array representing the vocabulary (idx2word).
    """
    print("\nÍndices:")
    for word in words_list:
        try:
            # Find the index of the word in the vocabulary
            idx = np.where(vocabulary_array == word)[0][0]
            print(f"  Palabra: '{word}', Índice: {idx}")
        except IndexError:
            print(f"  Palabra: '{word}' no encontrada en el vocabulario.")

# Llama a la función con las palabras seleccionadas y el vocabulario
print_word_indices(selected_words, idx2word)


Índices:
  Palabra: 'god', Índice: 8449
  Palabra: 'windows', Índice: 22084
  Palabra: 'people', Índice: 14666
  Palabra: 'problem', Índice: 15573
  Palabra: 'time', Índice: 20307


Se construye la matriz transpuesta

In [ ]:
# Se tiene X_train - documento × término
X_words = X_train.T  # término × documento


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_most_similar_words(word, X_words_matrix, vocabulary, top_n=5):
    """
    Calcula y devuelve las N palabras más similares a una palabra dada.

    Args:
        word (str): La palabra para la que se buscarán similitudes.
        X_words_matrix (csr_matrix): La matriz transpuesta (término x documento).
        vocabulary (np.ndarray): Array que mapea índices a palabras.
        top_n (int): Número de palabras más similares a devolver.

    Returns:
        list: Una lista de tuplas (palabra, similitud) de las palabras más similares.
    """
    try:
        word_idx = np.where(vocabulary == word)[0][0]
    except IndexError:
        return f"La palabra '{word}' no se encontró en el vocabulario."

    # Obtener el vector de la palabra de interés
    word_vector = X_words_matrix[word_idx]

    # Calcular la similitud coseno con todos los demás vectores de palabras
    similarities = cosine_similarity(word_vector, X_words_matrix)[0]

    # Excluir la palabra original (similaridad 1.0 consigo misma)
    # y obtener los índices de las palabras más similares en orden descendente
    most_similar_indices = np.argsort(similarities)[::-1][1:top_n + 1]

    results = []
    for idx in most_similar_indices:
        sim_score = similarities[idx]
        if sim_score > 0: # Solo considerar palabras con similitud > 0
            results.append((vocabulary[idx], sim_score))
    return results


# Bucle para cada palabra seleccionada
for word in selected_words:
    print(f"\nPalabras más similares a '{word}':")
    similar_words = get_most_similar_words(word, X_words, idx2word)

    if isinstance(similar_words, str):
        print(similar_words)
    elif similar_words:
        for sim_word, score in similar_words:
            print(f"  - {sim_word}: {score:.4f}")
    else:
        print("  No se encontraron palabras similares (con similitud > 0).")


Palabras más similares a 'god':
  - jesus: 0.2819
  - bible: 0.2775
  - christ: 0.2717
  - faith: 0.2594
  - existence: 0.2476

Palabras más similares a 'windows':
  - dos: 0.3014
  - microsoft: 0.2162
  - running: 0.1903
  - file: 0.1873
  - hierarchies: 0.1831

Palabras más similares a 'people':
  - don: 0.2423
  - think: 0.2295
  - government: 0.2013
  - just: 0.1879
  - like: 0.1765

Palabras más similares a 'problem':
  - fix: 0.1968
  - fixed: 0.1591
  - solve: 0.1444
  - using: 0.1381
  - problems: 0.1341

Palabras más similares a 'time':
  - like: 0.1681
  - long: 0.1612
  - don: 0.1577
  - just: 0.1512
  - think: 0.1462


En esta sección, se realizó nuevamente la similitud coseno, obteniendo valores bajos, pero relevantes.

- Para god, todos los terminos tienen una estrecha relación con la religión.
- Para windows, todos los terminos se asocian al software.
- Para people, los terminos son muy amplios, pero aun asi, tienen relacion al debate, opinion.
- Para problem, los terminos se relacionan claramente a los problemas.
- Para time, tambien existe una relacion mas general, pero menos evidente. Por ejemplo, long time, just in time, etc.

En conclusión, se permite identificar que tan similares son los terminos en funcion de los documentos que comparten.